In [5]:
# %load ProfileDomain.py

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division
import numpy as np
from scipy.optimize import minimize

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['savefig.dpi'] = 1.5 * matplotlib.rcParams['savefig.dpi']

import psrchive
from libstempo.libstempo import *
import libstempo as T

import corner as corner

import PTMCMCSampler
from PTMCMCSampler import PTMCMCSampler as ptmcmc

from Class import *

ImportError: No module named psrchive

In [ ]:
stylefilepath = os.getcwd().split('PALMCMC')[0]+'latex_stylefiles'
print stylefilepath

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
        'axes.labelsize': 10,
        'lines.markersize': 4,
        'font.size': 10,
        'xtick.major.size':6,
        'xtick.minor.size':3,  
        'ytick.major.size':6,
        'ytick.minor.size':3, 
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.2,
        'legend.fontsize': 7,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'savefig.dpi':200,
        'path.simplify':True,
        'font.family': 'serif',
        'font.serif':'Times',
        'text.latex.preamble': [r'\usepackage{amsmath}', 
                                r'\usepackage{'+stylefilepath+'/apjfonts}'],
        'text.usetex':True,
        'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                                'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
        #'font.serif':cm,
        'figure.figsize': (3.39,2.1)}
plt.rcParams.update(params)

In [ ]:
lfunc = Likelihood()
lfunc.loadPulsar("OneChan.par", "OneChan.tim", root='Sim1-OneChan')

## Get initial Fit to the Profile

In [ ]:
lfunc.TScrunch(doplot = True, channels = 1)

lfunc.getInitialParams(MaxCoeff = 20, cov_diag=[0.01, 0.1, 0.1], 
                       resume=True, outDir = './InitFFTMNChains/Max20-', 
                       sampler='multinest', incScattering = False, 
                       mn_live = 1000,  fitNComps = 1, doplot = True)

## Make interpolation Matrix

In [ ]:
lfunc.PreComputeFFTShapelets(interpTime = 1, MeanBeta = lfunc.MeanBeta, doplot=True)
lfunc.getInitialPhase(doplot = True)
lfunc.ScatterInfo = lfunc.GetScatteringParams(mode = 'parfile')

## Define parameter list and sampling ranges

In [ ]:
parameters = []
parameters.append('Phase')
for ii in range(lfunc.TotCoeff-1):
    for jj in range(lfunc.EvoNPoly+1):
        parameters.append('S'+str(ii+1)+'E'+str(jj))
for ii in range(lfunc.numTime):
    parameters.append(lfunc.psr.pars()[ii])
for ii in range(lfunc.NScatterEpochs):
    parameters.append("Scatter_"+str(ii))


print parameters
n_params = len(parameters)
print n_params
lfunc.n_params = n_params
    
pmin = np.array(np.ones(n_params))*-100
pmax = np.array(np.ones(n_params))*100

for i in range(lfunc.NScatterEpochs):
    pmin[-lfunc.NScatterEpochs+i] = -6
    pmax[-lfunc.NScatterEpochs+i] = 1

lfunc.pmin = pmin
lfunc.pmax = pmax

## Define starting point for sampling

In [ ]:
x0 = np.array(np.zeros(n_params))

pcount = 0
x0[pcount] = lfunc.MeanPhase
pcount += 1

for i in range(lfunc.TotCoeff-1):
    for j in range(lfunc.EvoNPoly+1):
        x0[pcount] = lfunc.MLShapeCoeff[1+i][j]
        pcount += 1


for ii in range(lfunc.numTime):
    x0[pcount+ii] = 0
pcount += lfunc.numTime
for ii in range(lfunc.NScatterEpochs):
    x0[pcount+ii] = lfunc.MeanScatter
pcount += lfunc.NScatterEpochs

In [ ]:
lfunc.calculateFFTHessian(x0)
covM = np.linalg.inv(lfunc.hess)
lfunc.PhasePrior = np.sqrt(covM[0,0]) * lfunc.ReferencePeriod
lfunc.MeanPhase = x0[0] * lfunc.ReferencePeriod

In [ ]:
lfunc.doplot = False
burnin=1000
sampler = ptmcmc.PTSampler(ndim=n_params, logl=lfunc.FFTMarginLogLike, logp=lfunc.my_prior,
                            cov=covM, outDir='./Chains/',resume=False)
sampler.sample(p0=x0, Niter=20000, isave=10, 
               burn=burnin, thin=1, neff=1000)

## Load MCMC chain

In [ ]:
chains = np.loadtxt('./Chains/chain_1.txt').T

## Make a plot

In [ ]:
chains = chains[:,burnin:]
if(lfunc.numTime > 0):
    Tchains = chains[1+lfunc.TotCoeff-1:1+lfunc.TotCoeff-1 + lfunc.numTime]
    figure = corner.corner(Tchains.T, labels=[r"$RA$", r"$DEC$", r"$F0$", r"$F1$"],
                       quantiles=[0.16, 0.5, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 12})

ML = chains.T[np.argmax(chains[-3])][:n_params]
lfunc.WaterFallPlot(ML)